In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install -q --upgrade transformers diffusers ftfy accelerate

In [6]:
# setup imports to get basic libraries

from base64 import b64encode

import torch
from diffusers import AutoencoderKL, LMSDiscreteScheduler, UNet2DConditionModel
from huggingface_hub import notebook_login

# video display to show transition
from IPython.display import HTML
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
from torch import autocast
from torchvision import transforms as tfms
from tqdm.auto import tqdm
from transformers import CLIPTextModel, CLIPTokenizer, logging

torch.manual_seed(1024)

# huggingface login - use token from huggingface user profile when prompted
# DO NOT HARDCODE OR SAVE IT
if not (Path.home()/'.cache/huggingface'/'token').exists():
    notebook_login()
    
# suppress warnings during CLIPTextModel
logging.set_verbosity_error()

# set default device to cuda when available
torch_device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
if "mps" == torch_device: os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = "1"

In [9]:
## code similar to huggingface example notebook

# these are the key components - model names upfront in case this needs to be migrated to other models
diffusion_model = "CompVis/stable-diffusion-v1-4"
text_tokenizer_model = "openai/clip-vit-large-patch14"

#1 Autoencoder to decode latents into images
vae = AutoencoderKL.from_pretrained(diffusion_model, subfolder = "vae")

#2 Tokenizer and Text Encoder for text
tokenizer = CLIPTokenizer.from_pretrained(text_tokenizer_model)
text_encoder = CLIPTextModel.from_pretrained(text_tokenizer_model)

#3 UNet model :)
unet = UNet2DConditionModel.from_pretrained(diffusion_model, subfolder = "unet")

#4 Noise scheduler
scheduler = LMSDiscreteScheduler(
    beta_start = 0.00085,
    beta_end = 0.012,
    beta_schedule = "scaled_linear",
    num_train_timesteps = 1000
)

# push to GPU
vae = vae.to(torch_device)
text_encoder = text_encoder.to(torch_device)
unet = unet.to(torch_device)

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

unet/config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]